Set system path so that pendulum module can be found.

In [1]:
import sys
sys.path.insert(1, '../')
print(sys.executable)
print(sys.version)

%matplotlib ipympl

/home/ms/prg/pendulum/venv/bin/python3
3.8.8 (default, Apr 13 2021, 19:58:26) 
[GCC 7.3.0]


Import packages

In [2]:
import numpy as np
from pendulum import controller, pendulum, sim, utils
from pendulum.viz import Visualizer
from IPython.display import HTML
import matplotlib.pyplot as plt

We set the timestep of the simulation (`dt`), the total simulation time (`t_final`), and create a pendulum. We set the pendulum to be in the upright position: 

$$ [x, \dot{x}, \theta, \dot{\theta}] = [0,0,0,0]$$

In [11]:
dt = 0.01
t_final = 15
pend = pendulum.Pendulum(2.0, 3.0, 2.0, initial_state=np.array([0,0,-np.pi,0]))

These parameters define our force function. We can define this function however we like; here, it is a sinusoidal impulse function that peaks at 4 seconds.

In [12]:
fig, ax = plt.subplots()
c1, c2, c3, c4 = 2, 3.0, 7, 2
fshift = 6
force_fn = lambda t: c1 * np.sin(c2*t) * c3/(c4*np.sqrt(np.pi)) * np.exp(-((t-fshift)/c4)**2)
fx = np.linspace(0,t_final, 500)
fy = force_fn(fx)
ax.plot(fx, fy)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Define the simulation with our parameters

In [13]:
simulation = sim.Simulation(dt, t_final, force_fn)

We create a new LQR controller. $Q$ has the weighted cost function over the inputs; here, we penalize error in $\theta$ highest. We also want to penalize control input, becuase it's assumed that we have finite actuation.

In [14]:
Q = [1,0,100,0]
R = 0.001
cont = controller.LQR_GPR(pend, dt, 7, 10, Q, R)

`simulate` actually runs the simulation. It calculates the pendulum state for each dt from t=0 to t=`t_final`.

In [15]:
results = simulation.simulate(pend, cont, plot=False)

100%|██████████| 1501/1501 [02:27<00:00, 10.15it/s]


We can create plots on the `results` dataframe returned by the simulation. For example, here is a plot of the states:

In [16]:
results[('lerr','t')] = results[('lpred','t')].shift(1, fill_value=0.0) - results[('state','t')]
results[('nlerr','t')] = results[('nlpred','t')].shift(1, fill_value=0.0) - results[('state','t')]

In [17]:
fig, ax = plt.subplots()
for s in results['state']:
    ax.plot(results[('state', s)], label=s)
    ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Here is a plot of the control actions over time.

In [18]:
fig, ax = plt.subplots()
ax.plot(results['lerr'], label='linear error')
ax.plot(results['nlerr'], label='nonlinear error')
ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

We can create an animation of our results using the `Visualizer` class:

In [19]:
visualizer = Visualizer(results, pend, speed=4)
anim = visualizer.animate((9.5, 5))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Some systems may have trouble displaying animations in the browser. We can prerender the animation as an HTML5 video which will display at full speed on all platforms (this cell may take a while to execute.) 

In [ ]:
HTML(anim.to_html5_video())